In [2]:
#paolo's version
import os
import platform
import dask
import distributed
import time
from dask.distributed import Client, LocalCluster, as_completed

###############################

def ping(event,actor_id):
    pid = os.getpid()
    hostname = platform.node()
    print(actor_id, pid, hostname,time.time(), " - started event ", event)
    time.sleep(event)
    print(actor_id, pid, hostname,time.time(), " - done with event ", event)

    return  (actor_id, event)

################################

#create cluster w/ number workers
cluster = LocalCluster(n_workers=5)
client = Client(cluster)

#create events for number of iterations
events = [e for e in range(10)]

#submit actors to ping function with args for id's
actors = [client.submit(ping, event=events.pop(), actor_id="actor" + str(a)) for a in range(5)]
print(actors)

################################

time.sleep(1)

################################

#waits for first actor to be done with their task using as_completed
seq = as_completed(actors)
print(seq)

#print actor, eliminate event of actor, return actor of said event back to ping function, 
#retire progam when all events are eliminated.
for future in seq:
    (actor, evtDone) = future.result()
    print(actor, " returned event ", evtDone)
    time.sleep(1)
    if events:
        event = events.pop() 
        new_future = client.submit(ping, event, actor)
        seq.add(new_future) # add back into the loop
        time.sleep(1)
    else:
        print("DONE: event list is empty")
        break
            
    

C:\Users\lists\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 59999 instead
  warnings.warn(


[<Future: pending, key: ping-8f82eac6a8e84070fa22339289a99f31>, <Future: pending, key: ping-ea794432829a39f974e0a0d0986ef891>, <Future: pending, key: ping-5ceb1883587a8d3a4da261ecbe50232a>, <Future: pending, key: ping-c7444eb2f7d6eb5f230923ef31f91552>, <Future: pending, key: ping-b0df416a389876261ccb3914f7f8929e>]
<as_completed: waiting=5 done=0>
actor4  returned event  15
actor3  returned event  16
actor2  returned event  17
actor1  returned event  18
actor0  returned event  19
actor0  returned event  10
actor2  returned event  12
actor1  returned event  11
actor3  returned event  13
actor4  returned event  14
actor4  returned event  5
actor3  returned event  6
actor1  returned event  7
actor2  returned event  8
actor0  returned event  9
actor0  returned event  0
DONE: event list is empty
